In [57]:
# imports 

import re
import sqlite3
import emoji

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from matplotlib.ticker import MaxNLocator
from datetime import datetime 

tqdm.pandas()

In [3]:
# paths 

path_to_db = '/Users/alexraudvee/Downloads/test.db'

path_to_conv = '/Users/alexraudvee/Desktop/TU:e/Q4Y1_data_challenge_1/conversations.json'

In [4]:
# connection to the database with sentiment scores

try:
    connection = sqlite3.connect(path_to_db)
    print('Succesfuly connected to the database')

    query = """
        SELECT name FROM sqlite_master  
        WHERE type='table';
        """

    cursor = connection.cursor()
    cursor.execute(query)
    list_of_tables = cursor.fetchall()

    print('---TABLE NAMES IN DB---')
    for table_name in list_of_tables:
        print('-----------------------')
        print(table_name[0])

    print('-----------------------')
    
except sqlite3.Error as e:
    print(e)


Succesfuly connected to the database
---TABLE NAMES IN DB---
-----------------------
users
-----------------------
tweets
-----------------------
tweets_geo
-----------------------
replies
-----------------------
retweets
-----------------------
quotes
-----------------------
hashtags
-----------------------
sqlite_sequence
-----------------------
symbols
-----------------------
user_mentions
-----------------------


In [93]:
# tweets table
def get_tweets(conn):
    query = """
        SELECT *
        FROM tweets
    """

    df = pd.read_sql(query, conn)
    return df

# users table
def get_users(conn):
    query = """
        SELECT *
        FROM users
    """

    df = pd.read_sql(query, conn)
    return df

# tweets geo table 
def get_tweets_geo(conn):
    query = """
        SELECT *
        FROM tweets_geo
    """

    df = pd.read_sql(query, conn)
    return df

# replies table
def get_replies(conn):
    query = """
        SELECT *
        FROM replies
    """

    df = pd.read_sql(query, conn)
    return df

# retweets table
def get_retweets(conn):
    query = """
        SELECT *
        FROM retweets
    """

    df = pd.read_sql(query, conn)
    return df

# quotes table
def get_quotes(conn):
    query = """
        SELECT *
        FROM quotes
    """

    df = pd.read_sql(query, conn)
    return df

# hastags table 
def get_hashtags(conn):
    query = """
        SELECT *
        FROM hashtags
    """

    df = pd.read_sql(query, conn)
    return df

# symbols table 
def get_symbols(conn):
    query = """
        SELECT *
        FROM symbols
    """

    df = pd.read_sql(query, conn)
    return df

# user mentions table 
def get_user_mentions(conn):
    query = """
        SELECT *
        FROM user_mentions
    """

    df = pd.read_sql(query, conn)
    return df

In [108]:
df_users = get_users(conn=connection)

In [109]:
df_users

,user_id,name,screen_name,location,followers_count,friends_count
0,22,rabble,rabble,PDX,16404,3182
1,155,Craig Calef,craigcalef,"BOS, SAN, SFO, LAS",2436,395
2,224,Dave Pell,davepell,San Francisco,54308,787
3,246,Blaine Cook,blaine,"London, UK",14968,2142
4,295,Josh Kopelman,joshk,Philly,132552,3795
...,...,...,...,...,...,...
2329595,1244677457028026370,Georgia,Georgia28337366,None,0,1
2329596,1244679780131311618,Flo,Flo43253301,None,0,0
2329597,1244685591809163264,Annie Beth,AnnieBe26893938,None,0,7
2329598,1244686973559803905,moons,M00nUn1t,None,0,0


In [94]:
# construct the dataframes for further analysis 

df_tweets = get_tweets(conn=connection)  
# df_users = get_users(conn=connection)
# df_replies = get_replies(conn=connection)
# df_retweets = get_retweets(conn=connection)
# df_quotes = get_quotes(conn=connection)
df_tweets_geo = get_tweets_geo(conn=connection)
# df_user_metions = get_user_mentions(conn=connection)
# df_symbols = get_symbols(conn=connection)
# df_hastags = get_hashtags(conn=connection)

In [95]:
# function to convert the sentiment label in numeric value

def convert_lable(lable):
    
    if lable == 'neutral':
        return int(0)
    elif lable == 'negative':
        return int(-1)
    elif lable == 'positive':
        return int(1)
    else:
        return None

In [96]:
# add new column with numeric values

df_tweets['sentiment_numeric'] = df_tweets['sentiment_label'].apply(lambda row: convert_lable(row))

In [102]:
df_tweets_geo = df_tweets_geo.rename(columns={'tweet_geo_id':'tweet_id'})

In [103]:
# construct the dataframe with conversations 

df_covs = pd.read_json(path_to_conv)

In [104]:
df_sentiment = df_tweets[['tweet_id', 'sentiment_numeric']] 

df_covs_sentiment = pd.merge(df_covs, df_sentiment, on='tweet_id')
df_covs_sentiment = pd.merge(df_covs_sentiment, df_tweets_geo, on='tweet_id')

In [105]:
df_covs_sentiment['respond_time'] = df_covs_sentiment['timestamp'].diff()

In [106]:
def convert_timedelta(datetime_obj):
    return datetime_obj.seconds / 60 / 60

In [107]:
df_covs_sentiment['respond_time'] = df_covs_sentiment['respond_time'].progress_apply(lambda row: convert_timedelta(row))

df_covs_sentiment

100%|██████████| 40961/40961 [00:00<00:00, 303990.89it/s]


,conv_id,user_id,tweet_id,text,timestamp,for_airline_id,sentiment_numeric,full_name,country,country_code,tweet_type,respond_time
0,20,21435317,1131180997909917696,@AmericanAir GA Esperanza AA2301 C31 in DFW ha...,2019-05-22 12:52:20.885,22536055,-1.0,"Grapevine, TX",United States,US,original,NaN
1,64,1013478272,1131198326462910467,@VirginAtlantic hi! Simple question. Why can w...,2019-05-22 14:01:12.334,20626359,-1.0,"Harwich, England",United Kingdom,GB,original,1.147500
2,64,1013478272,1131202636496154627,"@VirginAtlantic Oh well, we’re just cattle cla...",2019-05-22 14:18:19.926,20626359,-1.0,"Harwich, England",United Kingdom,GB,reply,0.285278
3,71,2859333782,1131199980796989442,"@AmericanAir listen, I work today. get it toge...",2019-05-22 14:07:46.758,22536055,-1.0,"Los Angeles, CA",United States,US,original,23.823889
4,83,56670181,1131203773685280770,Flying to NYC today @AmericanAir for the chris...,2019-05-22 14:22:51.053,22536055,0.0,Concourse C,United States,US,original,0.251111
...,...,...,...,...,...,...,...,...,...,...,...,...
40956,388210,430449595,1244646149623091207,@lufthansa pnr no JM998H from delhi to frankf...,2020-03-30 15:22:21.134,124476322,-1.0,"Bharatpur, India",India,IN,original,0.282222
40957,388213,1297050344,1244658494105681922,"@lufthansa Okay thank you Mac, so to be refund...",2020-03-30 16:11:24.288,124476322,0.0,"Dresden, Germany",Germany,DE,reply,0.817500
40958,388256,48374132,1244693180001521664,@VirginAtlantic Are you taking the Micky? Your...,2020-03-30 18:29:14.050,20626359,-1.0,"Nailsworth, England",United Kingdom,GB,reply,2.296944
40959,388268,1182693697605001216,1244671630116368387,"@British_Airways Hi there, I can’t get through...",2020-03-30 17:03:36.157,18332190,-1.0,"Camberwell, London",United Kingdom,GB,original,22.572778
